<a href="https://colab.research.google.com/github/HpDiniz/Leitor-de-PDF/blob/main/PDF_Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install pdfminer
!pip install PyPDF2

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2build2).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import io
import re
import os
import cv2
import PyPDF2
import pytesseract
import pandas as pd
from PIL import Image
from io import StringIO
from google.colab import files
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter

In [2]:
file_objects = files.upload()

Saving Extrato XP - Cliente 01.pdf to Extrato XP - Cliente 01 (14).pdf


In [3]:
# Converte o conteúdo do PDF para texto
def convert_pdf_to_txt(path):

    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()

    return text

def remove_header_and_footer(text, cm, tm, fontDict, fontSize):
    y = tm[5]
    if y > 50 and y < 720:
        parts.append(text)

def create_new_df():

    # Cria um DataFrame utilizado no consolidado final
    return pd.DataFrame({
        'CLASSIFICAÇÃO': pd.Series(dtype='str'),
        'ATIVO': pd.Series(dtype='str'),
        'EXPOSIÇÃO': pd.Series(dtype='str'),
        'INSTITUIÇÃO': pd.Series(dtype='str'),
        'ATUAL': pd.Series(dtype='str'),
        'DATA': pd.Series(dtype='str'),
        'ON/OFF': pd.Series(dtype='str'),
        'DATA DE VENCIMENTO': pd.Series(dtype='str')
    })

def obtem_classificacao(ativo):

    # Verifica se é uma ativo no formato de Ticker
    founded = re.search(r'\b[A-Z]{4}(3|4|5|6|11|32|33|34|35)\b', ativo, flags=(re.IGNORECASE))
    if founded:
        return "Ações"
    
    # Define as palavras chave para cada Classificação
    dict_classificacoes = {
        "Ações": [
            "FIC FIA","FIA","FI Ações", "FI Açoes"
        ],
        "Renda Fixa": [
            "Renda Fixa","RF","CDB","LC","LCA","LCI","CRA","CRI","Deb","Debenture",
            "Debênture","Tesouro","Pré","Pós","Pre","Pos","IPCA","IPCA+","LTN","NTN",
            "NTN-B","NTNB","DI","REF DI","CP","Credito Privado","Crédito Privado",
            "LP","Longo Prazo","Previdencia","Previdência","Prev","VGBL","PGBL"
        ],
        "Multimercado": [
            "FIC FIM","FIM","MM","Multimercado","Multi","COE","Multiestratégia",
            "Multiestratégia"
        ]
    }

    classes = []
    # Percorre cada uma das classificações
    for classificacao in dict_classificacoes:

        # Percorre as palavras-chave de cada uma das classificações
        for key in dict_classificacoes[classificacao]:

            # Verifica se a palavra-chave atual existe no nome do ativo
            founded = re.search(r'\b' + key + r'\b', ativo, flags=(re.IGNORECASE))
            if founded and not classificacao in classes:
                classes.append(classificacao)

    if len(classes) > 0:
        return ' / '.join(classes)

    # Se nenhuma classificação foi encontrada, verifica se é um ETF de investimento estrangeiro
    founded = re.search(r'\b[A-Z]{3}\b', ativo, flags=(re.IGNORECASE))
    if founded:
        return "Ações"

    return "***"

In [4]:
text_body = ""
text_miner = ""

for file in file_objects.keys():

    file_name, file_extension = os.path.splitext(file)

    if file_extension != '.pdf':
        continue

    # Lê o conteúdo do arquivo PDF
    text_miner = convert_pdf_to_txt(file)
    
    reader = PyPDF2.PdfReader(file)
    page_number = 0

    for page in reader.pages:

        parts = []
        page_number += 1

        page.extract_text(visitor_text=remove_header_and_footer)
        text_body = text_body + "".join(parts)

        if False:
            try:
                # Obtém os recursos da página
                xObject = page['/Resources']['/XObject'].get_object()

                image_number = 0
                # Percorre todos os objetos X da página
                for obj in xObject:

                    image_number += 1

                    # Verifica se o objeto é uma imagem
                    if xObject[obj]['/Subtype'] == '/Image':
                        size = (xObject[obj]['/Width'], xObject[obj]['/Height'])
                        data = xObject[obj].get_data()
                        if xObject[obj]['/ColorSpace'] == '/DeviceRGB':
                            mode = "RGB"
                        else:
                            mode = "P"

                        # Cria uma imagem a partir dos dados
                        img = Image.frombytes(mode, size, data)
                        image_name = file_name[0:25] + "_" + str(page_number) + "_" + str(image_number) + ".png"
                        #image_path = pdf_path = "/content/gdrive/My Drive/Temp/" + image_name

                        # Salva a imagem
                        img.save(image_name)

                        # Carrega a imagem
                        image = cv2.imread(image_name)

                        # Converte a imagem para escala de cinza
                        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

                        # Aplica uma suavização para remover ruídos
                        # image = cv2.medianBlur(image, 3)
                    
                        # Executa o OCR usando o Tesseract
                        text = pytesseract.image_to_string(image).strip()

                        print('Texto obtido para o arquivo "' + file_name + '": "' + text + '"')

                        #files.download(image_name)
            except:
                print('Não foi possível ler as imagens do arquivo "' + file_name + '"')

In [5]:
print(text_miner)

Extrato de Cotista

Consolidado

Movimentação de 15/05/2022 a 11/11/2022

Nome do Cliente
Endereço

CEP: 02440-00

POSIÇÃO CONSOLIDADA

Fundo

Cota

Quantidade

Valor Aplicado

Valor Bruto

IR

SulAmérica Inflatie FI Renda Fixa Longo Prazo

XP Referenciado FI Referenciado DI CP

XPA MM II FIC FIM

XPA RV FIC FIM

XPA Vértice Crédito FIM CP

49,03907260

2,55168782

1,48495549

1,51471586

1,31704969

485,46020696

9.607,86926679

206.015,91600648

86.244,99799744

62.558,82325043

21.434,59

23.303,62

231.165,15

119.600,00

67.664,32

23.806,52

24.516,28

305.924,47

130.636,67

82.393,08

13,80

272,84

2.095,63

1.655,50

816,74

Emissão: 11/11/2022

Código: 3Nxxx7

IOF

0,00

0,00

0,00

0,00

0,00

Valor Liquido

23.792,72

24.243,44

303.828,84

128.981,17

81.576,34

Total na Instituição

463.167,68

567.277,02

4.854,51

0,00

562.422,51

SulAmérica Inflatie FI Renda Fixa Longo Prazo

Data Mov.

Cotização

Histórico

Cota

Quantidade

Valor Bruto

15/05/2022
25/05/2022
04/08/

In [6]:
# Regex para obter a data de emissão
regex_0 = "(?<=Movimenta..o de \d{2}\/\d{2}\/\d{4} a )\d{2}\/\d{2}\/\d{4}"
data_emissao = str(re.search(regex_0, text_miner, flags=(re.IGNORECASE)).group(0))

# Regex para obter o texto situado entre o termo "POSIÇÃO CONSOLIDADA" e o termo "Emissão"
regex_1 = "(?<=POSI..O CONSOLIDADA).*?(?=Emissão:)"
result_1 = str(re.search(regex_1, text_miner, flags=(re.IGNORECASE | re.DOTALL)).group(0))

# Regex para obter todas as linhas que possuírem pelo menos uma letra e um espaço
regex_2 = "^.*[a-zA-Z] .*$"
result_2 = re.findall(regex_2, result_1, flags=(re.MULTILINE))

# Gera um dicionário de termos com espaço
dictionary = {}
print("TEXTOS A SEREM CORRIGIDOS:")
for result in result_2:
    dictionary[result.replace(" ","")] = result
    print(" - " + result)

TEXTOS A SEREM CORRIGIDOS:
 - Valor Aplicado
 - Valor Bruto
 - SulAmérica Inflatie FI Renda Fixa Longo Prazo
 - XP Referenciado FI Referenciado DI CP
 - XPA MM II FIC FIM
 - XPA RV FIC FIM
 - XPA Vértice Crédito FIM CP


In [7]:
# Regex para obter o texto situado entre o termo "POSIÇÃOCONSOLIDADA" e o termo "TotalnaInstituição"
regex_3 = "(?<=POSI..OCONSOLIDADA\n).*?(?=TotalnaInstituição)"
result_3 = str(re.search(regex_3, text_body, flags=(re.IGNORECASE | re.DOTALL)).group(0))

# Substitui os espaços por um ponto e vírgula, para simular uma tabela CSV
result_3 = result_3.replace(" ",";")
print(result_3)

Fundo;Cota;Quantidade;ValorAplicado;ValorBruto;IR;IOF;ValorLiquido
SulAméricaInflatieFIRendaFixaLongoPrazo;49,03907260;485,46020696;21.434,59;23.806,52;13,80;0,00;23.792,72
XPReferenciadoFIReferenciadoDICP;2,55168782;9.607,86926679;23.303,62;24.516,28;272,84;0,00;24.243,44
XPAMMIIFICFIM;1,48495549;206.015,91600648;231.165,15;305.924,47;2.095,63;0,00;303.828,84
XPARVFICFIM;1,51471586;86.244,99799744;119.600,00;130.636,67;1.655,50;0,00;128.981,17
XPAVérticeCréditoFIMCP;1,31704969;62.558,82325043;67.664,32;82.393,08;816,74;0,00;81.576,34



In [8]:
# Altera todos os textos incorretos para a versão correta
for key in dictionary:
    if key in result_3:
        result_3 = result_3.replace(key,dictionary[key])

print(result_3)

Fundo;Cota;Quantidade;Valor Aplicado;Valor Bruto;IR;IOF;ValorLiquido
SulAmérica Inflatie FI Renda Fixa Longo Prazo;49,03907260;485,46020696;21.434,59;23.806,52;13,80;0,00;23.792,72
XP Referenciado FI Referenciado DI CP;2,55168782;9.607,86926679;23.303,62;24.516,28;272,84;0,00;24.243,44
XPA MM II FIC FIM;1,48495549;206.015,91600648;231.165,15;305.924,47;2.095,63;0,00;303.828,84
XPA RV FIC FIM;1,51471586;86.244,99799744;119.600,00;130.636,67;1.655,50;0,00;128.981,17
XPA Vértice Crédito FIM CP;1,31704969;62.558,82325043;67.664,32;82.393,08;816,74;0,00;81.576,34



In [9]:
# Converte o CSV para o tipo DataFrame
df = pd.read_csv(StringIO(result_3), sep=";")
df

,Fundo,Cota,Quantidade,Valor Aplicado,Valor Bruto,IR,IOF,ValorLiquido
0,SulAmérica Inflatie FI Renda Fixa Longo Prazo,"49,03907260","485,46020696","21.434,59","23.806,52","13,80","0,00","23.792,72"
1,XP Referenciado FI Referenciado DI CP,"2,55168782","9.607,86926679","23.303,62","24.516,28","272,84","0,00","24.243,44"
2,XPA MM II FIC FIM,"1,48495549","206.015,91600648","231.165,15","305.924,47","2.095,63","0,00","303.828,84"
3,XPA RV FIC FIM,"1,51471586","86.244,99799744","119.600,00","130.636,67","1.655,50","0,00","128.981,17"
4,XPA Vértice Crédito FIM CP,"1,31704969","62.558,82325043","67.664,32","82.393,08","816,74","0,00","81.576,34"


In [10]:
# Cria o DataFrame que conterá o consolidado final
df_result = create_new_df()

# Preenche o consolidado final
for index, row in df.iterrows():

    df_result = df_result.append({
        'CLASSIFICAÇÃO': obtem_classificacao(row['Fundo']),
        'ATIVO': row['Fundo'],
        'EXPOSIÇÃO': "Real",
        'INSTITUIÇÃO': "XP Investimentos",
        'ATUAL': row["Valor Bruto"],
        'DATA': data_emissao,
        'ON/OFF': "ON",
        'DATA DE VENCIMENTO': "-"
    }, ignore_index=True)

df_result

,CLASSIFICAÇÃO,ATIVO,EXPOSIÇÃO,INSTITUIÇÃO,ATUAL,DATA,ON/OFF,DATA DE VENCIMENTO
0,Renda Fixa,SulAmérica Inflatie FI Renda Fixa Longo Prazo,Real,XP Investimentos,"23.806,52",11/11/2022,ON,-
1,Renda Fixa,XP Referenciado FI Referenciado DI CP,Real,XP Investimentos,"24.516,28",11/11/2022,ON,-
2,Multimercado,XPA MM II FIC FIM,Real,XP Investimentos,"305.924,47",11/11/2022,ON,-
3,Multimercado,XPA RV FIC FIM,Real,XP Investimentos,"130.636,67",11/11/2022,ON,-
4,Renda Fixa / Multimercado,XPA Vértice Crédito FIM CP,Real,XP Investimentos,"82.393,08",11/11/2022,ON,-


In [11]:
print("Disponibilizando arquivo de consolidado...")

for file in file_objects.keys():

    file_name, file_extension = os.path.splitext(file)

    df_result.to_excel(file_name + ".xlsx", index=False) 
    files.download(file_name + ".xlsx")

Disponibilizando arquivo de consolidado...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>